In [ ]:
!pip install openai
!pip install langchain

In [20]:
import os
os.environ["OPENAI_API_KEY"] = "my key"

#1.Prompt

In [22]:
from langchain.llms import OpenAI

llm = OpenAI(model_name="text-davinci-003")

prompt = """
今天是星期二, 明天是星期四.

上面的描述存在什么问题?
"""
llm(prompt)

'\n描述中缺少了明天是星期三的内容。'

#2.Prompt Template

In [23]:
from langchain.llms import OpenAI
from langchain import PromptTemplate

llm = OpenAI(model_name="text-davinci-003")

#下面的{location}作为替换符
#加入了Instruction，让LLM知道提问的背景。让它知道调用旅游方面的能力。
template = """
你作为导游帮我回答如下问题，回应不超过2句话。
我想去{location}旅游，有什么景点可以推荐的?

"""

prompt = PromptTemplate(
    #接受用户输入
    input_variables=["location"],
    #定义Prompt tempalte
    template=template,
)
#这里是真正的用户输入 "北京"
final_prompt = prompt.format(location='北京')

print (f"组合后的用户请求: {final_prompt}")

print (f"大语言模型的回应: {llm(final_prompt)}")

组合后的用户请求: 
你作为导游帮我回答如下问题，回应不超过2句话。
我想去北京旅游，有什么景点可以推荐的?


大语言模型的回应: 
北京有许多精彩景点，如故宫、天安门广场、颐和园等。


#3.Output Parser

格式指令 - 一个自动生成的提示，告诉LLM如何根据你期望的结果格式化它的响应

解析器 - 一种将模型的文本输出提取到期望的结构（通常是json）的方法

In [24]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI

llm = OpenAI(model_name="text-davinci-003")

#定义输出结构
response_schemas = [
    ResponseSchema(name="错误的文字", description="这里是输入错误的问题"),
    ResponseSchema(name="正确的文字", description="这里是纠正以后正确的文字")
]

#定义输出解析器
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

#获取输出指令
format_instructions = output_parser.get_format_instructions()
print (format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"错误的文字": string  // 这里是输入错误的问题
	"正确的文字": string  // 这里是纠正以后正确的文字
}
```


In [25]:
template = """
你扮演一个小学语文老师，帮助同学修改错别字。
将用户输入的错别字进行修改，保证输出的文字是正确的。

用户输入:
{user_input}

以 {format_instructions} 的格式输出
"""

prompt = PromptTemplate(
    input_variables=["user_input"],
    partial_variables={"format_instructions": format_instructions},
    template=template
)

promptValue = prompt.format(user_input="今天的天其真好啊！")

print(promptValue)


你扮演一个小学语文老师，帮助同学修改错别字。
将用户输入的错别字进行修改，保证输出的文字是正确的。

用户输入:
今天的天其真好啊！

以 The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"错误的文字": string  // 这里是输入错误的问题
	"正确的文字": string  // 这里是纠正以后正确的文字
}
``` 的格式输出



In [26]:
#大语言模型得到的结果
llm_output = llm(promptValue)
#返回给用户的结果
output_parser.parse(llm_output)


{'错误的文字': '天其', '正确的文字': '天气'}